In [1]:
# goals
# is winrate dependent on time spent in last game?
# is winrate dependent on accuracy?
# is winrate dependent on accuracy of game before, if this game is a reque?
# is winrate dependent on # of reques?

# how to find time spent in a game?
# how do you determine if a game is a reque?

import archives_manager
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.width', 10000)

In [2]:
start_unix = 1696132800
end_unix = 1698724800
player_name = "BIG_TONKA_T"

games_archive = archives_manager.get_games_between_timestamps(
    player_name=player_name,
    start_unix=start_unix,
    end_unix=end_unix,
    time_class='rapid',
    filter_func=archives_manager.build_archive_filter(rated=True)
)

archives_manager.simplified_archived_game(games_archive[0])

{'url': 'https://www.chess.com/game/live/89923795267',
 'end_time': 1696185098,
 'date': '2023.10.01',
 'rated': True,
 'time_class': 'rapid',
 'white': {'username': 'Quarini', 'rating': 1064, 'result': 'resigned'},
 'black': {'username': 'BIG_TONKA_T', 'rating': 1089, 'result': 'win'}}

In [3]:
archives_manager.simplified_archived_game(games_archive[-1])

{'url': 'https://www.chess.com/game/live/92035707081',
 'end_time': 1698296732,
 'date': '2023.10.26',
 'rated': True,
 'time_class': 'rapid',
 'white': {'username': 'Primodiurno', 'rating': 1456, 'result': 'resigned'},
 'black': {'username': 'BIG_TONKA_T', 'rating': 1494, 'result': 'win'}}

In [5]:
games_data = []

unix_range = end_unix - start_unix

for game in games_archive:
    opp_games_archive = None
    opp_name = archives_manager.get_opponent_name(game, player_name)

    # log progress
    print(opp_name, round((game['end_time'] - start_unix) / (unix_range), 2))
    
    try:
        opp_games_archive = archives_manager.get_games_between_timestamps(
            player_name=opp_name,
            start_unix=game['end_time'] - 30 * 24 * 60 * 60,
            end_unix=game['end_time'],
            time_class='rapid',
            filter_func=archives_manager.build_archive_filter(rated=True)
        )
    except archives_manager.ArchiveRetrievalError as e:
        print(f'{opp_name} archive failed')

    if opp_games_archive is None:
        continue

    games_data.append({
        'game': game,
        'opp_name': opp_name,
        'opp_games_archive': opp_games_archive
    })

Quarini 0.02
hgarciarod 0.02
HERFJERF 0.02
olas109 0.02
InkLv 0.02
alamrini 0.02
djgreen1 0.02
Pytoo91 0.02
mumergaddafi 0.02
Copertus 0.02
CyborgCerberus94 0.03
Nadirmitch 0.03
Rensa0 0.03
wimperneel 0.03
nightjumper39 0.03
Spiderbreeze 0.03
milton2000 0.03
ahmedtaha09 0.03
Godofspeeds 0.03
Krop_duster 0.03
nickringgod 0.03
8akk3 0.03
lilchernobyl 0.03
SamtheTimTamMan 0.03
unicorndash1 0.03
Robari1 0.03
DrimoDramo 0.03
Hoaiarc 0.03
ochuckster123 0.03
SirKennard 0.03
Rick_woote 0.03
Callsign_Gimli 0.03
zzyix 0.03
RedscareRAM 0.03
Alex-Sokol 0.03
ejonokuchi 0.03
EasyRider3202 0.03
instant_10 0.03
Ysulai 0.03
JohannFehrmann 0.03
Ek3498 0.03
georgemetcalf22 0.03
Rotorhead64 0.03
defeatlazydoor 0.03
karthikpan 0.03
OrangPeeler 0.03
jackpotbrandon 0.03
Pseudonym20 0.03
GeorgeLamponi 0.03
aleksandrstrilnyk 0.03
Deejai1220 0.03
morethancipher 0.04
ClarkGriswald 0.04
ToplessChess11 0.05
gokselur 0.05
yellowmoose 0.05
P3rs0Nwastaken 0.05
nikhil6hama 0.05
Wickrama 0.05
gardemarine 0.05
Ametaka 0

In [43]:
tyler_rows = []
opp_data = []

for data in games_data:
    game = data['game']
    opp_name = data['opp_name']
    opp_games_archive = data['opp_games_archive']

    # analyze the opponent's history (for opp_data)
    opp_game_rows = []
    for opp_game in opp_games_archive:
        # get elo of the 'Player' from the perspective of player_name=opp_name, to get that historical elo rating
        opp_game_elo = archives_manager.get_elo(opp_game, player_name=opp_name)

        opp_game_rows.append({
            'unix': opp_game['end_time'],
            'elo': opp_game_elo['Player'], # 'Player' elo from the perspective of player_name=opp_name
            'elo_diff': opp_game_elo['Player'] - opp_game_elo['Opponent'], # Difference between opponent's rating, and opponent's oppenent rating
            # accuracy
            # average eval change per move, per game state
            # opening classification
            # average time spent per move, per game state
            'color': archives_manager.get_color(opp_game, opp_name),
            'won': archives_manager.get_won(opp_game, opp_name)
        })
    opp_data.append(opp_game_rows)

    # analyze this game (for tyler_rows)
    elo = archives_manager.get_elo(game, player_name)
    acc = archives_manager.get_accuracy(game, player_name)
    color = archives_manager.get_color(game, player_name)

    tyler_rows.append({
        # basic info
        'url': game['url'],
        'unix': game['end_time'],
        'player_elo': elo['Player'],
        'opp_elo': elo['Opponent'],
        'opp_name': opp_name,

        # predictors
        'opp_30d': len(opp_games_archive),
        'elo_diff': elo['Player'] - elo['Opponent'],
        'color': color,
        # time of day as 'cos_24h', 'sin_24h'

        # game review
        'player_acc': None if acc is None else acc['Player'],
        'opp_acc': None if acc is None else acc['Opponent'],
        'acc_diff': None if acc is None else acc['Player'] - acc['Opponent'],
        # player pieces left
        # opp pieces left
        # move count
        # average eval change per move, per game state
        # average time spent per move, per game state
        # opening classification

        # targets
        'won': archives_manager.get_won(game, player_name)
    })

In [50]:
# for each opponent history data, fix the elo since we know the true value based on the associated game in the player history

# this output highlights the issue
print(tyler_rows[0]) # tyler's least recent game (beginning of player data)
print(opp_data[0][-1]) # for that game, the opponent's most recent game in history is the same game but with guessed elo

{'url': 'https://www.chess.com/game/live/89923795267', 'unix': 1696185098, 'player_elo': 1089, 'opp_elo': 1064, 'opp_name': 'Quarini', 'opp_30d': 249, 'elo_diff': 25, 'color': False, 'player_acc': 69.43, 'opp_acc': 62.25, 'acc_diff': 7.180000000000007, 'won': 1}
{'unix': 1696185098, 'elo': 1063, 'elo_diff': -27, 'color': True, 'won': 0}


In [52]:
for i in range(len(tyler_rows)):
    opp_elo = tyler_rows[i]['opp_elo']
    opp_data[i][-1]['elo'] = opp_elo

print(tyler_rows[0]) # tyler's least recent game (beginning of player data)
print(opp_data[0][-1]) # for that game, the opponent's most recent game in history is the same game but with guessed elo

{'url': 'https://www.chess.com/game/live/89923795267', 'unix': 1696185098, 'player_elo': 1089, 'opp_elo': 1064, 'opp_name': 'Quarini', 'opp_30d': 249, 'elo_diff': 25, 'color': False, 'player_acc': 69.43, 'opp_acc': 62.25, 'acc_diff': 7.180000000000007, 'won': 1}
{'unix': 1696185098, 'elo': 1064, 'elo_diff': -27, 'color': True, 'won': 0}


In [ ]:
# turn tyler_rows into tyler_df and make even more predictors

# ma5, ma20, x-ma5, x-ma20, d_x-ma5, d_x-ma20
# is_reque
# num_reque
# prev_avg_eval_change _open _mid _end
# prev_time_spent _open _mid _end
# prev_acc
# prev_acc_diff
# prev_ player opp _pieces_left